In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, Dropout, Bidirectional, Input, GRU
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import optimizers 
import re
import tensorflow as tf 
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import os
from google.colab import drive 

In [2]:
drive.mount('/content/drive')
df_path = "/content/drive/MyDrive/AIM_Task/preprocessed_data.csv"
df = pd.read_csv(df_path)

Mounted at /content/drive


In [3]:
df

,dialect,text
0,10,اللي حابين يشاركون بالمسابقة
1,10,هل تعلم أنك تشاهد خشمك طوال الوقت لكن عقلك تبر...
2,11,شو رح يقول الحريري بدي ك قلبو لسمير ججعج بس ارجع
3,13,صح صح اذكر كانو يعبون لنا بترول ببلاش
4,9,اساسا ماعندج القدره على الرد ثانيا شو انجازا...
...,...,...
458192,13,اللي يحط دعايات في اغاني اليهال ف يوتيوب الله ...
458193,17,قرف دى ملبن بالمكسرات
458194,10,حنيتو لشتاء ايه الله يعين اندفنا من الغبار و س...
458195,13,بتنتظر كثير لانهم ماهم بكفو يسونها السعايده ...


In [4]:
df_copy = df.copy()

In [6]:
X = df_copy['text'].values
X_test = X[:20000]
X_train = X[20000:]

# y = pd.get_dummies(df_copy['dialect']).values
y = df_copy['dialect']
y_test = y[:20000]
y_train = y[20000:]

In [7]:
t = Tokenizer(oov_token='<UNK>')
t.fit_on_texts(X_train)

In [8]:
max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']

(('ملوعتبي', 461828), ('<PAD>', 0), 1)

In [9]:
train_sequences = t.texts_to_sequences(X_train)
test_sequences = t.texts_to_sequences(X_test)

In [10]:
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 1000

X_train = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((438197, 1000), (20000, 1000))

In [11]:
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight('balanced',
                                                   classes = np.unique(df_copy['dialect']),
                                                   y = df_copy['dialect'])

In [12]:
class_weights = dict(enumerate(class_weights))

In [13]:
class_weights

{0: 2.7531244742471217,
 1: 2.56425797208511,
 2: 2.2060307556017755,
 3: 1.76357135159269,
 4: 1.6426010769109434,
 5: 1.5729709503113694,
 6: 1.5672570427834558,
 7: 1.3316273743925973,
 8: 0.968180012509086,
 9: 0.9680327383970524,
 10: 0.9486951732591268,
 11: 0.9217289672625154,
 12: 0.9116933092972633,
 13: 0.8193179339176957,
 14: 0.6974270223537327,
 15: 0.604511835685694,
 16: 0.5819438729113641,
 17: 0.441657798752323}

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
def train_model(model, data, targets):
    text_clf = Pipeline([
    ('clf', model),
    ])
    text_clf.fit(data, targets)
    return text_clf
def get_accuracy(trained_model,X, y):
    predicted = trained_model.predict(X)
    accuracy = np.mean(predicted == y)
    return accuracy

In [21]:
from sklearn.naive_bayes import MultinomialNB
trained_clf_multinomial_nb = train_model(MultinomialNB(), X_train, y_train)
accuracy = get_accuracy(trained_clf_multinomial_nb,X_test, y_test)
print(accuracy)

0.06245


In [22]:
from sklearn.linear_model import LogisticRegression
trained_clf_LogisticRegression = train_model(LogisticRegression(class_weight=class_weights), X_train, y_train)
accuracy = get_accuracy(trained_clf_LogisticRegression,X_test, y_test)
print(accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.03655


In [ ]:
from sklearn.linear_model import SGDClassifier
trained_clf_SGDClassifier = train_model(SGDClassifier(class_weight=class_weights), X_train, y_train)
accuracy = get_accuracy(trained_clf_SGDClassifier,X_test, y_test)
print(accuracy)

In [ ]:
from sklearn.svm import LinearSVC
trained_clf_LinearSVC = train_model(LinearSVC(), X_train, y_train)
accuracy = get_accuracy(trained_clf_LinearSVC,X_test, y_test)
print(accuracy)